In [1]:
import mne
import scipy.io
import numpy as np
import os
import pandas as pd
import queue
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import messagebox

In [2]:
class CrossSubject_Panel:
    def __init__(self,w, Database):
        self.w =w 
        self.data = Database.data

        self.CrossFile = tk.LabelFrame(self.w, text="Cross Validation File", height=150, bg='White', width=200, labelanchor='n')
        self.Rangeframe = tk.LabelFrame(self.w, text="Selected Range", height=150, bg='White', width=200, labelanchor='n')
        self.Trainframe = tk.LabelFrame(self.w, text="Train", height=150, bg='White', width=200, labelanchor='n')
        self.Valframe = tk.LabelFrame(self.w, text="Validation", height=150, bg='White', width=200, labelanchor='n')
        self.Testframe = tk.LabelFrame(self.w, text="Test", height=150, bg='White', width=200, labelanchor='n')
        self.autosplitvar = tk.IntVar()
        self.autosplitbutton = tk.Checkbutton(self.w, text='Auto split all files by formats above.',var=self.autosplitvar, onvalue=1, offvalue=0,  bg='White') 
        
        self.CrossFile.grid(row=0, column=0,padx=10,pady=5, ipady=4,sticky='w')
        self.Rangeframe.grid(row=0, column=1,padx=10,pady=5,  ipady=4,sticky='w')
        self.Trainframe.grid(row=1, column=0,padx=10,pady=5,  ipady=4,sticky='w')
        self.Valframe.grid(row=1, column=1,padx=10,pady=5, ipady=4,sticky='w')
        self.Testframe.grid(row=1, column=2,padx=10,pady=5, ipady=4,sticky='w')
        self.autosplitbutton.grid(row=2, column=0,columnspan=2,padx=10,ipady=4,sticky='w')
        tk.Button(self.w, text="Confirm", bg= 'White', command=self.Input2Database).grid(row=3, column=1,padx=15,pady = 10, ipady=2,sticky=tk.W)

        '''crossfile frame'''
        self.FileValue = tk.IntVar() 
        NoneButton = tk.Radiobutton(self.CrossFile, text='None',variable= self.FileValue, value=1, bg= 'White', command = self.LOSOstate) 
        LOSOButton = tk.Radiobutton(self.CrossFile, text='Leave-one-subject-out',variable= self.FileValue, value=2, bg= 'White', command = self.LOSOstate)
        NoneButton.grid(row=0, column=0,padx=20,ipady=2,sticky=tk.W)
        LOSOButton.grid(row=1, column=0,padx=20,ipady=2,sticky=tk.W) 
        
        '''rangeframe'''
        tk.Label(self.Rangeframe, text="Excluded Class", bg= 'White').grid(row=0, column=0,padx=10,ipady=4,sticky=tk.W)
        tk.Label(self.Rangeframe, text="Onset Time(ms)", bg= 'White').grid(row=1, column=0,padx=10,ipady=4,sticky=tk.W)
        self.data.ExcludedClass= tk.Entry(self.Rangeframe) 
        self.data.ExcludedClass.insert(0, 'ex: 1')
        self.data.SelectedOnset= tk.Entry(self.Rangeframe)
        self.data.SelectedOnset.insert(0, 'ex: -100~3000')
      
        self.data.ExcludedClass.grid(row=0, column=1, padx=10,ipady=4,sticky=tk.W)
        self.data.SelectedOnset.grid(row=1, column=1, padx=10,ipady=4,sticky=tk.W)
        

        ''' frame of train/val/test'''
        self.files= ['S01_1.set', 'S01_2.set']

        '''Trainframe'''
        tk.Label(self.Trainframe, text="Choose files ", bg= 'White').grid(row=0, column=0,padx=10,ipady=2,sticky=tk.W)
        #box
        self.TrainFilesbox= self.CreateListbox(self.Trainframe, self.files, row=1)
        self.TrainFilesbox.grid(row=1, column=0,rowspan=2,columnspan=2, padx=10,ipady=4,sticky=tk.W)
        self.TrainFilesbox['state']= 'disabled'
        
        #button
        self.TrainConfirmButton= tk.Button(self.Trainframe, text="Confirm", bg= 'White', command=lambda:self.ResetorConfirm(self.TrainFilesbox, 'Train', reset=False))
        self.TrainResetButton= tk.Button(self.Trainframe, text="Reset", bg= 'White', command=lambda:self.ResetorConfirm(self.TrainFilesbox,'Train', reset= True))
        self.TrainConfirmButton.grid(row=3, column=0,padx=10,pady = 5, ipady=2,sticky=tk.W)
        self.TrainResetButton.grid(row=3, column=1,padx=5,pady = 5, ipady=2,sticky=tk.W)

        '''valframe'''
        tk.Label(self.Valframe, text="File: ", bg= 'White').grid(row=0, column=0,padx=10,ipady=2,sticky=tk.W)
        tk.Label(self.Valframe, text="Train/Val ratio: ", bg= 'White').grid(row=1, column=0,padx=10,ipady=2,sticky=tk.W)
        tk.Label(self.Valframe, text="Choose files: ", bg= 'White').grid(row=2, column=0,padx=10,ipady=2,sticky=tk.W)
        #box
        self.ValFilebox = ttk.Combobox(self.Valframe, values=['None','Split from Train set','Choose a file'])
        self.ValFilesbox= self.CreateListbox(self.Valframe, self.files, row=3 )
        self.TrainValratiobox= tk.Entry(self.Valframe)

        self.ValFilebox.grid(row=0, column=1,padx=10,ipady=2,sticky=tk.W)
        self.TrainValratiobox.grid(row=1, column=1, padx=10,ipady=4,sticky=tk.W)
        self.ValFilesbox.grid(row=2, column=1,rowspan=2,columnspan=2,padx=10,ipady=2,sticky=tk.W)
        self.ValFilebox['state']='disabled'
        self.ValFilesbox['state']='disabled'
        self.TrainValratiobox['state']='disabled'
        self.ValFilebox.bind('<<ComboboxSelected>>', lambda x :self.state('val'))

        #button
        self.ValConfirmButton= tk.Button(self.Valframe, text="Confirm", bg= 'White', command=lambda:self.ResetorConfirm(self.ValFilesbox, 'Val', reset=False))
        self.ValResetButton= tk.Button(self.Valframe, text="Reset", bg= 'White', command=lambda:self.ResetorConfirm(self.ValFilesbox,'Val', reset= True))
        self.ValConfirmButton.grid(row=5, column=1,padx=10,pady = 5, ipady=2,sticky=tk.W)
        self.ValResetButton.grid(row=5, column=2,padx=0,pady = 5, ipady=2,sticky=tk.W)
      

        

        '''Testframe'''
        tk.Label(self.Testframe, text="File: ", bg= 'White').grid(row=0, column=0,padx=10,ipady=2,sticky=tk.W)
        tk.Label(self.Testframe, text="Train/Test ratio: ", bg= 'White').grid(row=1, column=0,padx=10,ipady=2,sticky=tk.W)
        tk.Label(self.Testframe, text="Choose files: ", bg= 'White').grid(row=2, column=0,padx=10,ipady=2,sticky=tk.W)
        
        #box
        self.TestFilebox = ttk.Combobox(self.Testframe, values=['None','Split from Train set', 'Choose a file'])
        self.TestFilesbox= self.CreateListbox(self.Testframe, self.files, row=2)
        self.TrainTestratiobox= tk.Entry(self.Testframe) 

        self.TestFilebox.grid(row=0, column=1,padx=10,ipady=2,sticky=tk.W)
        self.TrainTestratiobox.grid(row=1, column=1, padx=10,ipady=4,sticky=tk.W)
        self.TestFilesbox.grid(row=2, column=1,rowspan=2,columnspan=2, padx=10,ipady=2,sticky=tk.W)
        self.TestFilebox.bind('<<ComboboxSelected>>', lambda x : self.state('test'))

        self.TestFilebox['state']='disabled'
        self.TestFilesbox['state']='disabled'
        self.TrainTestratiobox['state']= 'disabled'

        #button
        self.TestConfirmButton= tk.Button(self.Testframe, text="Confirm", bg= 'White', command=lambda:self.ResetorConfirm(self.TestFilesbox, 'Test', reset=False))
        self.TestResetButton= tk.Button(self.Testframe, text="Reset", bg= 'White', command=lambda:self.ResetorConfirm(self.TestFilesbox,'Test', reset= True))
        self.TestConfirmButton.grid(row=4, column=1,padx=10,pady = 5, ipady=2,sticky=tk.W)
        self.TestResetButton.grid(row=4, column=2,padx=5,pady = 5, ipady=2,sticky=tk.W)
       
    def ResetorConfirm(self,box,filetype, reset):
        if reset ==False:
            box['state']= 'disabled'
            
            if filetype == 'Train':
                self.data.TrainFile= [self.files[idx] for idx in box.curselection()]
                self.TrainConfirmButton['state']= 'disabled'
                self.TrainResetButton['state']= 'normal'
                print(self.data.TrainFile)
            elif filetype =='Val':
                self.data.ValFile= box.curselection()
                self.ValConfirmButton['state']= 'disabled'
                self.ValResetButton['state']= 'normal'
                
            elif filetype == 'Test':
                self.data.TestFile= box.curselection()
                self.TestConfirmButton['state']= 'disabled'
                self.TestResetButton['state']= 'normal'
        else:
            box['state']= 'normal'

            if filetype == 'Train':
                self.data.TrainFile= None
                print(self.data.TrainFile)
                self.TrainConfirmButton['state']= 'normal'
                self.TrainResetButton['state']= 'disabled'
            elif filetype =='Val':
                self.data.ValFile= None
                self.ValConfirmButton['state']= 'normal'
                self.ValResetButton['state']= 'disabled'
            elif filetype == 'Test':
                self.data.TestFile= None
                self.TestConfirmButton['state']= 'normal'
                self.TestResetButton['state']= 'disabled'
        
    def CreateListbox(self, frame, files, row):
        box = tk.Listbox(frame, selectmode=tk.EXTENDED)
        [box.insert(tk.END, file) for file in files]

        sb = tk.Scrollbar(frame, orient=tk.VERTICAL)
        sb.grid(row=row,column=2,sticky=tk.N+tk.S+tk.W)
        box.config(yscrollcommand=sb.set)
        sb.config(command=box.yview)

        return box
    
    def LOSOstate(self):
        if int(self.FileValue.get()) == 1 : #none
           self.TrainFilesbox['state']= 'normal'
           self.ValFilebox['state']='readonly'
           self.TestFilebox['state']='readonly'
        
           self.TrainConfirmButton['state']= 'normal'
           self.ValConfirmButton['state']= 'normal'
           self.TestConfirmButton['state']= 'normal'
           

        elif int(self.FileValue.get()) == 2 : #LOSO
            self.TrainFilesbox['state']= 'disabled'
            self.ValFilebox['state']='disabled'
            self.ValFilesbox['state']='disabled'
            self.TrainValratiobox['state']='disabled' 
            self.TestFilebox['state']='disabled'
            self.TestFilesbox['state']='disabled'
            self.TrainTestratiobox['state']= 'disabled'

            self.TrainConfirmButton['state']= 'disabled'
            self.TrainResetButton['state']= 'disabled'
            self.ValConfirmButton['state']= 'disabled'
            self.ValResetButton['state']= 'disabled'
            self.TestConfirmButton['state']= 'disabled'
            self.TestResetButton['state']= 'disabled'

    def state(self, box):
        
        if box == 'val' :
            if str(self.ValFilebox.get())== 'Choose a file':
                self.ValFilesbox['state']= 'normal'
                self.TrainValratiobox['state']= 'disable'

            elif str(self.ValFilebox.get())=='Split from Train set':
                self.TrainValratiobox['state']= 'normal'
                self.ValFilesbox['state']= 'disabled'

        elif box == 'test':
            if str(self.TestFilebox.get())== 'Choose a file':
                self.TestFilesbox['state']= 'normal' 
                self.TrainTestratiobox['state']= 'disable'

            elif str(self.TestFilebox.get())=='Split from Train set':
                self.TrainTestratiobox['state']= 'normal' 
                self.TestFilesbox['state']= 'disable' 
      
    def Input2Database(self):
        self.data.ExcludedClass= self.CheckEmptyValues(self.data.ExcludedClass)
        self.data.SelectedOnset= self.CheckEmptyValues(self.data.SelectedOnset)
        self.data.autosplit= int(self.autosplitvar.get())
        self.CheckFile('Train')
        self.CheckFile('Val')
        self.CheckFile('Test')

        print('Trainfile: ', self.data.TrainFile)
        print('Valfile: ',self.data.ValFile)
        print('Testfile: ', self.data.TestFile)
        print('TrainValRatio:',self.data.TrainValratio)
        print('TrainTestratio: ',self.data.TrainTestratio)
        print(f'excluded class: {self.data.ExcludedClass}')
        print('kfold: ', self.data.kfold)
        print('autosplit: ', self.data.autosplit)
        self.w.destroy()

        # '''Data Sort and Split'''
        # DataSortandSplit(self.data)
    
    def CheckFile(self, filetype):
        if filetype == 'Train':
            self.data.TrainFile=self.data.TrainFile
            
        if filetype =='Val':
            box = self.ValFilebox
            
            if str(box.get()) == 'Split from Train set':
                self.data.TrainValratio= int(self.TrainValratiobox.get())

            elif str(box.get()) == 'Choose a file':
                self.data.ValFile=self.data.ValFile

        elif filetype =='Test':
            box = self.TestFilebox
            if str(box.get()) == 'Split from Train set':
                self.data.TrainTestratio= int(self.TrainTestratiobox.get())

            elif str(box.get()) == 'Choose a file':
                self.data.TestFile= self.data.TestFile

    def CheckEmptyValues(self, value):

        #check empty values
        if len(value.get()) != 0:
                temp= str(value.get())
                value= self.CompleteNumber(temp)
                
        else:
                messagebox.showerror("Error",f'You must enter a number.', parent=self.w)
                
        return temp #str
    
    def CompleteNumber(self,string): # for 1~5 -> 1,2,3,4,5
        # if onsetorclass ='onset':
        #     if '~' in string: 
        #         string= string.split('~')
        #         strList = [i for i in range(int(string[0]),int(string[1]))] 
        #         #strList= [sub for sub in self.AllSub if sub >= string[0] and sub <= string[1]]

        if ',' in string:
            
            intList= [int(i) for i in string.split(',')]
            output= intList

        else:
            strList= string
            output= string 
        
        return output # string or intlist
            

In [7]:
from database import Database

COLOR= 'White'
w = tk.Tk()
w.geometry('950x500')
w.title("ExBrainable")
w.configure(background=COLOR)

d= Database()


'''define frame and notebook style'''
noteStyler = ttk.Style()
noteStyler.configure("TNotebook", background=COLOR, borderwidth=0)
noteStyler.configure("TNotebook.Tab", background=COLOR, foreground='Black', lightcolor=COLOR, borderwidth=0)
noteStyler.configure("TFrame", background=COLOR, foreground=COLOR, borderwidth=0)

'''create tabs'''
TabParent=ttk.Notebook(w, style= "TNotebook")
Tab1= ttk.Frame(TabParent, style='TFrame')
Tab2= ttk.Frame(TabParent, style='TFrame')
TabParent.add(Tab1, text= 'Individual Subject')
TabParent.add(Tab2, text= 'Cross Subject')
TabParent.grid(row=0, column=0)

CrossSubject_Panel(Tab2, d)
w.mainloop()

In [5]:
'''
class Individual_Panel
- state():防呆
- Input2Database(): transfer entry values to database
    + checkemptyvalues(): empty entry show error
        completeNumber(): string to list ex: class= '1,2,3,4' ->  ['1']
    + Checkfile(): identify val/test belongs to choices of 
                 split from train or choose a file

'''


class Individual_Panel:
    def __init__(self,w, Database):
        self.w =w 
        self.data = Database.data

        self.CrossFile = tk.LabelFrame(self.w, text="Cross Validation File", height=10, bg='White', width=50, labelanchor='n')
        self.Rangeframe = tk.LabelFrame(self.w, text="Selected Range", height=10, bg='White', width=50, labelanchor='n')
        self.Trainframe = tk.LabelFrame(self.w, text="Train", height=10, bg='White', width=50, labelanchor='n')
        self.Valframe = tk.LabelFrame(self.w, text="Validation", height=10, bg='White', width=50, labelanchor='n')
        self.Testframe = tk.LabelFrame(self.w, text="Test", height=10, bg='White', width=50, labelanchor='n')
        self.autosplitvar = tk.IntVar()
        self.autosplitbutton = tk.Checkbutton(self.w, text='Auto split all files by formats above.', var=self.autosplitvar, onvalue=1, offvalue=0,  bg='White') 

        
        self.CrossFile.grid(row=0, column=0,padx=20,pady=5, ipady=4,sticky=tk.W)
        self.Rangeframe.grid(row=1, column=0,padx=20,pady=5,  ipady=4,sticky=tk.W)
        self.Trainframe.grid(row=2, column=0,padx=20,pady=5,  ipady=4,sticky=tk.W)
        self.Valframe.grid(row=3, column=0,padx=20,pady=5, ipady=4,sticky=tk.W)
        self.Testframe.grid(row=4, column=0,padx=20,pady=5, ipady=4,sticky=tk.W)
        self.autosplitbutton.grid(row=5, column=0, padx=20,ipady=4,sticky=tk.W)
        

        # self.crossFile
        self.FileValue = tk.IntVar() 
        NoneButton = tk.Radiobutton(self.CrossFile, text='None',variable= self.FileValue, value=1, bg= 'White', command = self.kfoldstate) 
        kFoldButton = tk.Radiobutton(self.CrossFile, text='k fold',variable= self.FileValue, value=2, bg= 'White', command = self.kfoldstate)
        NoneButton.grid(row=0, column=0,padx=20,ipady=2,sticky=tk.W)
        kFoldButton.grid(row=1, column=0,padx=20,ipady=2,sticky=tk.W) 
        
            # input k
        tk.Label(self.CrossFile, text="k =", bg= 'White').grid(row=1, column=1,padx=20,ipady=4,sticky=tk.W)
        self.data.kfold= tk.Entry(self.CrossFile) 
        self.data.kfold.grid(row=1, column=2, padx=20,ipady=4,sticky=tk.W)

        # range frame: class onset
        tk.Label(self.Rangeframe, text="Excluded Class", bg= 'White').grid(row=0, column=0,padx=20,ipady=4,sticky=tk.W)
        tk.Label(self.Rangeframe, text="Onset Time(ms)", bg= 'White').grid(row=1, column=0,padx=10,ipady=4,sticky=tk.W)
        self.data.ExcludedClass= tk.Entry(self.Rangeframe) 
        self.data.ExcludedClass.insert(0, 'ex: 1')
        self.data.SelectedOnset= tk.Entry(self.Rangeframe)
        self.data.SelectedOnset.insert(0, 'ex: -100~3000')
      
        self.data.ExcludedClass.grid(row=0, column=1, padx=20,ipady=4,sticky=tk.W)
        self.data.SelectedOnset.grid(row=1, column=1, padx=20,ipady=4,sticky=tk.W)
        

        # data frame: train val test 
        tk.Label(self.Trainframe, text="Choose a file ", bg= 'White').grid(row=0, column=0,padx=20,ipady=2,sticky=tk.W)
        tk.Label(self.Valframe, text="File: ", bg= 'White').grid(row=0, column=0,padx=20,ipady=2,sticky=tk.W)
        tk.Label(self.Valframe, text="Choose a file: ", bg= 'White').grid(row=1, column=0,padx=20,ipady=2,sticky=tk.W)
        tk.Label(self.Valframe, text="Train/Val ratio: ", bg= 'White').grid(row=2, column=0,padx=20,ipady=2,sticky=tk.W)
        tk.Label(self.Testframe, text="File: ", bg= 'White').grid(row=0, column=0,padx=20,ipady=2,sticky=tk.W)
        tk.Label(self.Testframe, text="Choose a file: ", bg= 'White').grid(row=1, column=0,padx=20,ipady=2,sticky=tk.W)
        tk.Label(self.Testframe, text="Train/Test ratio: ", bg= 'White').grid(row=2, column=0,padx=20,ipady=2,sticky=tk.W)

        self.TrainFilesbox = ttk.Combobox(self.Trainframe, values=['S01_1.set', 'S03_1.set'],state="readonly")
        self.ValFilebox = ttk.Combobox(self.Valframe, values=['None','Split from Train set','Choose a file'],state="readonly")
        self.ValFilesbox = ttk.Combobox(self.Valframe, values=['S01_1.set'],state="readonly")
        self.TrainValratiobox= tk.Entry(self.Valframe) 
        self.TestFilebox = ttk.Combobox(self.Testframe, values=['None','Split from Train set', 'Choose a file'],state="readonly")
        self.TestFilesbox = ttk.Combobox(self.Testframe, values=['S01_1.set'],state="readonly")
        self.TrainTestratiobox= tk.Entry(self.Testframe) 
        
        

        self.TrainFilesbox.grid(row=0, column=1,padx=20,ipady=2,sticky=tk.W)
        self.ValFilebox.grid(row=0, column=1,padx=20,ipady=2,sticky=tk.W)
        self.ValFilesbox.grid(row=1, column=1,padx=20,ipady=2,sticky=tk.W)
        self.TrainValratiobox.grid(row=2, column=1, padx=20,ipady=4,sticky=tk.W)
        self.TestFilebox.grid(row=0, column=1,padx=20,ipady=2,sticky=tk.W)
        self.TestFilesbox.grid(row=1, column=1,padx=20,ipady=2,sticky=tk.W)
        self.TrainTestratiobox.grid(row=2, column=1, padx=20,ipady=4,sticky=tk.W)
        
        self.ValFilebox['state']= 'disabled'
        self.ValFilesbox['state']= 'disabled'
        self.TestFilesbox['state']= 'disabled'
        self.TrainValratiobox['state']= 'disabled'
        self.TrainTestratiobox['state']= 'disabled'

        
        self.ValFilebox.bind('<<ComboboxSelected>>', lambda x :self.state('val'))
        self.TestFilebox.bind('<<ComboboxSelected>>', lambda x : self.state('test'))
        
        tk.Button(self.w, text="Confirm", bg= 'White', command= self.Input2Database).grid(row=6, column=0,padx=100,pady = 30, ipady=2,sticky=tk.W)

    def kfoldstate(self):
        if int(self.FileValue.get()) == 1 : #none
            self.ValFilebox['state']= 'readonly'

        elif int(self.FileValue.get()) == 2 : #kfold
            self.ValFilebox['state']= 'disabled'
           
    def state(self, box):
       
        if box == 'val' :
            if str(self.ValFilebox.get())== 'Choose a file':
                self.ValFilesbox['state']= 'readonly'
                self.TrainValratiobox['state']= 'disable'

            elif str(self.ValFilebox.get())=='Split from Train set':
                self.TrainValratiobox['state']= 'normal'
                self.ValFilesbox['state']= 'disabled'

        elif box == 'test':
            if str(self.TestFilebox.get())== 'Choose a file':
                self.TestFilesbox['state']= 'readonly' 
                self.TrainTestratiobox['state']= 'disable'

            elif str(self.TestFilebox.get())=='Split from Train set':
                self.TrainTestratiobox['state']= 'normal' 
                self.TestFilesbox['state']= 'disable' 
      
                
                
    def Input2Database(self):
        self.data.ExcludedClass= self.CheckEmptyValues(self.data.ExcludedClass)
        self.data.SelectedOnset= self.CheckEmptyValues(self.data.SelectedOnset)
        self.data.kfold= self.CheckEmptyValues(self.data.kfold)
        self.data.autosplit= int(self.autosplitvar.get())
        self.CheckFile('Train')
        self.CheckFile('Val')
        self.CheckFile('Test')
        print(self.data.ValFile, self.data.TestFile, self.data.TrainValratio, self.data.TrainTestratio, self.data.kfold)
        print('autosplit: ', self.data.autosplit)
        self.w.destroy()
                
    def CheckEmptyValues(self, value):

        #check empty values
        if len(value.get()) != 0:
                temp= str(value.get())
                value= self.CompleteNumber(temp)
                
        else:
                messagebox.showerror("Error",f'You must enter a number.', parent=self.w)
                
        return temp #str
    
    def CompleteNumber(self,string): # for 1~5 -> 1,2,3,4,5
        # if onsetorclass ='onset':
        #     if '~' in string: 
        #         string= string.split('~')
        #         strList = [i for i in range(int(string[0]),int(string[1]))] 
        #         #strList= [sub for sub in self.AllSub if sub >= string[0] and sub <= string[1]]

        if ',' in string:
            
            intList= [int(i) for i in string.split(',')]
            output= intList

        else:
            strList= string
            output= string 
        
        return output # string or intlist

        
    def CheckFile(self, filetype):
        if filetype == 'Train':
            self.data.TrainFile=str(self.TrainFilesbox.get())
            
        if filetype =='Val':
            box = self.ValFilebox
            
            if str(box.get()) == 'Split from Train set':
                self.data.TrainValratio= int(self.TrainValratiobox.get())

            elif str(box.get()) == 'Choose a file':
                self.data.ValFile=str(self.ValFilesbox.get())

        elif filetype =='Test':
            box = self.TestFilebox
            if str(box.get()) == 'Split from Train set':
                self.data.TrainTestratio= int(self.TrainTestratiobox.get())

            elif str(box.get()) == 'Choose a file':
                self.data.TestFile=str(self.TestFilesbox.get())
            
        print(filetype,' ', str(box.get()))


        

In [39]:
from database import Database

COLOR= 'White'
w = tk.Tk()
w.geometry('400x650')
w.title("ExBrainable")
w.configure(background=COLOR)

d= Database()


'''define frame and notebook style'''
noteStyler = ttk.Style()
noteStyler.configure("TNotebook", background=COLOR, borderwidth=0)
noteStyler.configure("TNotebook.Tab", background=COLOR, foreground='Black', lightcolor=COLOR, borderwidth=0)
noteStyler.configure("TFrame", background=COLOR, foreground=COLOR, borderwidth=0)

'''create tabs'''
TabParent=ttk.Notebook(w, style= "TNotebook")
Tab1= ttk.Frame(TabParent, style='TFrame')
Tab2= ttk.Frame(TabParent, style='TFrame')
TabParent.add(Tab1, text= 'Individual Subject')
TabParent.add(Tab2, text= 'Cross Subject')
TabParent.grid(row=0, column=0)

Individual_Panel(Tab1, d)
w.mainloop()

In [31]:
print(d.data.ValFile, d.data.TestFile, d.data.TrainValratio, d.data.TrainTestratio, d.data.kfold)

None S01_1.set 5 None 4


In [18]:
'''
class DataSplit
- Info (should move to load data)
  DataInfo(): stack info of each file
  MakeInfo(): sub/session/onset/class of event -> pd.dataframe
- Sort(): split data based on choices of 1. None 2. Split from train 3. choose a file
    Sortbycriteria(): convert filename into sub/session intlist  2. filtering onset/class/session
- Split(): kfold, train/val ,train/test split
    SplitData(): sklearn normal and stratified kfold
'''
from sklearn.model_selection import KFold, StratifiedKFold

class DataSortandSplit:
    def __init__(self,data):
        self.data= data

        self.data.ExcludedClass  = ['1','2'] # mne event num 1~n, eventdict revise
        self.data.ExcludedClass = [int(i) for i in self.data.ExcludedClass] #revised ok
        self.data.SelectedOnset = '-100~3000'
        self.data.TrainFile = ['S03_1.set']
        #self.data.ValFile = ['S03_1.set'] 
        #self.data.ValFile = None
        self.data.TestFile = ['S03_2.set']
        #self.data.TestFile = None
        self.data.TrainValratio = None
        self.data.TrainTestratio = None
        self.data.kfold= 4

        print(f'kfold: {self.data.kfold}')
        print(f'excluded class: {self.data.ExcludedClass}')
        print(f'TrainValratio: {self.data.TrainValratio}')
        print(f'TrainTestratio: {self.data.TrainTestratio}')
        
        self.data.Info= self.datainfo()
        self.main()
    
    def main(self):

        if self.data.kfold == None:
            print('no kfold')
            self.Train()
            self.Test()
            self.Validation()
        else: 
            print('kfold split')
            
            self.Train()
            self.Test()

            '''kfold'''
            ratio= self.data.kfold
            iterate = True
            self.data.kfoldTrainValidx=self.SplitData(self.data.TrainData._data, self.data.TrainLabel,ratio= ratio, kfold='Stratified', iterate= iterate)
            print('fold 1 test',self.data.TrainLabel[self.data.kfoldTrainValidx[1]])
    

       
    def Train(self):
        '''Train'''
        self.data.TrainData, self.data.TrainLabel, self.Trainsub, self.Trainsess= self.Sortbycriteria(self.data.TrainFile[0]) # onset, class
        
        print('Trainsub/sess: ',self.Trainsub, self.Trainsess)
        print('Train seleted data: ',self.data.TrainData)   
        print('shape',self.data.TrainData._data.shape)
        print('label shape', self.data.TrainLabel.shape)

    def Test(self):
        '''Test'''
        # split from train
        if self.data.TrainTestratio != None: 
            print('split from train')
            self.Testsub = self.Trainsub
            self.Testsess= self.Trainsess
            self.data.TrainData, self.data.TestData, self.data.TrainLabel, self.data.TestLabel=self.SplitfromTrain(self.data.TrainTestratio)
            
            print('Test seleted data: ',self.data.TestData)   
            print('shape',self.data.TestData._data.shape)
            print('label shape', self.data.TestLabel.shape)
         # choose test file
         
        elif self.data.TestFile != None: 
            print('choose test file')    
            self.data.TestData, self.data.TestLabel, self.Testsub, self.Testsess =self.Sortbycriteria(self.data.TestFile[0])

        
            
            print('Test seleted data: ',self.data.TestData)   
            print('shape',self.data.TestData._data.shape)
            print('label shape', self.data.TestLabel.shape)

        else: 
            print('No test data and labels')
        
    def Validation(self):
        '''Validation'''
        # split from train
        if self.data.TrainValratio != None: 
            print('split from train')
            self.Valsub = self.Trainsub
            self.Valsess= self.Trainsess
            self.data.TrainData, self.data.ValData, self.data.TrainLabel, self.data.ValLabel=self.SplitfromTrain(self.data.TrainValratio)

            print('Val seleted data: ',self.data.ValData)   
            print('shape',self.data.ValData._data.shape)
            print('label shape', self.data.ValLabel.shape)

        # choose val file 
        elif self.data.ValFile != None: 
            print('choose val file')    
            self.data.ValData, self.data.ValLabel,self.Valsub, self.Valsess= self.Sortbycriteria(self.data.ValFile[0])
            
        
            print('Val seleted data: ',self.data.ValData)   
            print('shape',self.data.ValData._data.shape)
            print('label shape', self.data.ValLabel.shape)

        else:
            print('No val data and labels')
        
    def SplitfromTrain(self, ratio):
        iterate= False
        TrainTestidx=self.SplitData(self.data.TrainData._data, self.data.TrainLabel,ratio= ratio, kfold='Stratified', iterate= iterate)
        traindata= self.data.TrainData[TrainTestidx[0]]
        testdata= self.data.TrainData[TrainTestidx[1]]
        trainlabel= self.data.TrainLabel[TrainTestidx[0]]
        testlabel= self.data.TrainLabel[TrainTestidx[1]]

        return traindata, testdata, trainlabel, testlabel

    def Sortbycriteria(self, Filename):
        
        '''split file string into sublist, sessionlist'''# intlist
        file= Filename.split('_') # revise: cross subject
        SelectedSub = [int(i) for i in file[0] if i.isdigit()]
        SelectedSub = [j for j in SelectedSub if j>0]
        SelectedSession = [int(s) for s in file[1] if s.isdigit()]
        
        

        '''filtering subject/session/class/onset(revised)'''
        include= self.data.Info['Subject'].isin(SelectedSub)
        SelectedData= self.data.Info[include]
        #print(SelectedData)
        include= SelectedData['Session'].isin(SelectedSession)
        
        SelectedData= SelectedData[include]
        
        mask= SelectedData['Class'].isin(self.data.ExcludedClass)
        SelectedData= SelectedData[~mask]

        # onsettime

        '''select array by index'''
        index= SelectedData.index
        SelectedData= self.data.Data[Filename][index]
        SelectedLabel= np.asarray(SelectedData.events[:,2])
    
        return SelectedData, SelectedLabel, SelectedSub, SelectedSession # dataframe 
        #SelectedSub, SelectedSession : list .ex: [3]

    def datainfo(self):
        # ready when loading data 
        path= 'G:/我的雲端硬碟/109_2 course/Project/ExBrainable/GUI/xydata/S03/train_epoch.set'
        raw = mne.io.read_epochs_eeglab(path, uint16_codec='latin1') 
        path= 'G:/我的雲端硬碟/109_2 course/Project/ExBrainable/GUI/xydata/S03/test_epoch.set'
        raw_t = mne.io.read_epochs_eeglab(path, uint16_codec='latin1') 
        
        self.data.Data= {}
        self.data.Data['S03_1.set']= raw
        self.data.Data['S03_2.set']= raw_t

        
        k=0
        for file in self.data.Data.keys():

            files= file.split('_') # revise: cross subject
            sub = [int(i) for i in files[0] if i.isdigit()]
            sub = [j for j in sub if j>0]
            session = [int(s) for s in files[1] if s.isdigit()]
            

            df= self.MakeInfo(self.data.Data[file], sub[0], session[0])
            k+=1
            if 'Info' not in locals(): #revised 
                Info= df
            else:
                temp= df
                Info= pd.concat([Info, temp])

        # each file has a raw in Data , all file events in info

        return Info

    def MakeInfo(self,raw, sub,session): # sub,session : int
        onset= np.diff(raw.events[:,0])
        # insert length of last trial, make sure order is (trial, ch, tp)
        onset= np.insert(onset, len(raw.events)-1 , raw._data[-1,-1,:].shape , axis=0) 
        data={ 'Subject': [sub]* len(raw.events),
            'Session': [session]* len(raw.events), 
            'Class': raw.events[:,2],
            'Onset time':  onset}
        df= pd.DataFrame(data)

        return df

       
    '''iterate= False -> 只有kfold第一份，拿來當單純的split
    iterate= True -> 完整的kfold
    '''
    def SplitData(self,data, label, ratio, kfold='Stratified', iterate= True):
        TrainTestidx= [] # even: Train, odd: Test

        if kfold == 'Stratified':
            kf = StratifiedKFold(n_splits=ratio, shuffle= True, random_state= 1)
            split= kf.split(data,label)
        elif kfold == 'normal':
            kf = KFold(n_splits=ratio, shuffle= True,random_state= 1)
            split = kf.split(data)

        for trainidx, testidx in split:
            TrainTestidx.append(trainidx)
            TrainTestidx.append(testidx)
            if iterate ==False: # train/val split只需要第一份
                break

        
        return TrainTestidx
       

          

ImportError: DLL load failed while importing _ufuncs: 找不到指定的模組。

NameError: name 'd' is not defined

In [2]:
from database import Database
d= Database()

S= DataSortandSplit(d.data)


NameError: name 'DataSortandSplit' is not defined

In [32]:
from sklearn.model_selection import KFold, StratifiedKFold
'''iterate= False -> 只有kfold第一份，拿來當單純的split
   iterate= True -> 完整的kfold
'''
def SplitData(data, label, ratio, kfold='Stratified', iterate= True):
    TrainTestidx= [] # even: Train, odd: Test

    if kfold == 'Stratified':
        kf = StratifiedKFold(n_splits=ratio, shuffle= True, random_state= 1)
        split= kf.split(data,label)
    elif kfold == 'normal':
        kf = KFold(n_splits=ratio, shuffle= True,random_state= 1)
        split = kf.split(data)

    for trainidx, testidx in split:
        TrainTestidx.append(trainidx)
        TrainTestidx.append(testidx)
        if iterate ==False: # train/val split只需要第一份
            break

    
    return TrainTestidx

'''
split train/test before train/val
train/test has label must be stratified kfold
train/val can be stratified kfold or normal kfold
'''
d.data.TrainTestratio= 3
d.data.TrainValratio = 4
d.data.TestData= None
d.data.TestLabel= None

def Split(kfold = True):

    if kfold ==True: 
        iterate= True
    else:
        iterate= False

    # with TestData
    if d.data.TestData != None: 
        if len(d.data.TestLabel) == 0:
            print('train/test without testlabel: kfold')
            TrainTestidx=SplitData(d.data.TrainData._data,None, ratio= d.data.TrainTestratio, kfold='normal', iterate= iterate)
        else: 
            print('train/test with testlabel: stratifiedkfold')
            TrainTestidx=SplitData(d.data.TrainData._data, d.data.TrainLabel, ratio= d.data.TrainTestratio, kfold='Stratified', iterate=iterate)
        
        if kfold ==True:
            print('train/val with trainlabel and kfold: 把每一份stratified kfold裡面的train拿出來切val')
            TrainValidx=[]
            for n in range(d.data.TrainTestratio): 
                idx= TrainTestidx[2*n]
                afoldidx=SplitData(d.data.TrainData._data[idx], d.data.TrainLabel[idx],ratio= d.data.TrainValratio, kfold='Stratified', iterate= False)
                TrainValidx.append(afoldidx)
        
        else:
            print('train/val with trainlabel without kfold')
            idx= TrainTestidx[0]
            TrainValidx=SplitData(d.data.TrainData._data[idx], d.data.TrainLabel[idx],ratio= d.data.TrainValratio, kfold='Stratified', iterate= False)
    #without TestData
    else:
        print('train/val without testdata: stratifiedkold')
        TrainValidx=SplitData(d.data.TrainData._data, d.data.TrainLabel,ratio= d.data.TrainValratio, kfold='Stratified', iterate= iterate)
        TrainTestidx= None
  
    return TrainTestidx, TrainValidx

d.data.kfoldTrainTestidx, d.data.kfoldTrainValidx=Split(kfold = False)
print(d.data.kfoldTrainTestidx)
#print(d.data.kfoldTrainValidx[1][0])
#print(d.data.TrainLabel[d.data.kfoldTrainValidx[1][0]])


train/val without testdata: stratifiedkold
None


In [29]:

    
 # event_id, selection, events
############### ready while loading data 


In [30]:
# schedule for all subjects



     Subject  Session  Class  Onset time
0          3        1      1         562
3          3        1      1         562
5          3        1      1         562
7          3        1      3         562
9          3        1      4         562
..       ...      ...    ...         ...
283        3        1      3         562
284        3        1      4         562
285        3        1      3         562
286        3        1      1         562
287        3        1      3         562

[216 rows x 4 columns]


In [ ]:
class AutoGroupFiles:
    def __init__(self,data):
        self.files = ['S03_1.set',  'S03_2.set','S01_1.set', 'S02_1.set','S05_2.set']

        allsubsession=[]
        for file in self.files:
            sub, session=self.SplitSubSession(file)
            allsubsession.append((sub[0],session[0]))
        print(allsubsession)
        print(allsubsession.sort(key=lambda x: x[0]))
        
        

    def displaygroups(self):
        table= ttk.Treeview(w, height=10)
        sb = tk.Scrollbar(w, orient=tk.VERTICAL)
        sb.pack(side=tk.RIGHT, fill=tk.Y)
        table.config(yscrollcommand=sb.set)
        sb.config(command=table.yview)

        table['column']= ['Train','Validation', 'Test']
        table.column('#0', width=0, stretch=tk.NO)
        table.column('Train', anchor=tk.CENTER, width=50 )
        table.column('Validation', anchor=tk.CENTER, width=50)
        table.column('Test', anchor=tk.CENTER, width=50)
        table.heading('#0', text='', anchor=tk.CENTER)
        table.heading('Train', text='Train', anchor=tk.CENTER)
        table.heading('Validation', text='Validation', anchor=tk.CENTER)
        table.heading('Test', text='Test', anchor=tk.CENTER)

        item= table.insert(parent='', index=row, iid= row, text='', values=[id, type])
        #self.data.Label = ['S03_1.set', 'S01_1.set', 'S02_1.set','S05_2.set']
        
    def main(self):
        


        '''Train'''
        
        '''Test'''
        if self.data.TrainTestratio != None: 
            print('split from train')

        elif self.data.TestFile != None: 
            print('choose test file')    

        else:
            print('No test data and labels')
    def SplitSubSession(self, Filename):
        '''split file string into sublist, sessionlist'''# intlist
        file= Filename.split('_') # revise: cross subject
        SelectedSub = [int(i) for i in file[0] if i.isdigit()]
        SelectedSub = [j for j in SelectedSub if j>0]
        SelectedSession = [int(s) for s in file[1] if s.isdigit()]

        return SelectedSub, SelectedSession
